# Amazon Commerce Reviews

**Kaggle: 184.702 TU ML WS 20**

**Goal: Predict who wrote the review.**

### Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

### Get the Data

In [ ]:
data = pd.read_csv('./184702-tu-ml-ws-20-amazon-commerce-reviews/amazon_review_ID.shuf.lrn.csv')

In [ ]:
data

### Basic Data Information

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include = 'object')

### Check missing values

In [ ]:
print(data.isnull().sum(axis=0))

In [ ]:
sns.heatmap(data.isnull(), yticklabels=False, cbar=False, cmap='viridis')

In [ ]:
# Remove the target and check if the data is all int64

### Exploratory Data Analysis

**Class**

In [ ]:
plt.figure(figsize=(20, 8))
plt.xticks(rotation=90)
sns.set_style('darkgrid')
ax = sns.countplot(x = data['Class'])

total = len(data['Class'])

for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x()+p.get_width()/2.,
            height + 3,
            '{:.1f}%'.format(100 * height/total),
            ha="center")